In [1]:
import numpy as np
import pandas as pd

In [2]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 1CCF-D498

 C:\Users\S2민정S2\Desktop\빅콘\embedding 디렉터리

2020-08-30  오후 09:42    <DIR>          .
2020-08-30  오후 09:42    <DIR>          ..
2020-08-30  오후 04:51    <DIR>          .ipynb_checkpoints
2020-08-30  오후 09:35    <DIR>          __pycache__
2020-08-30  오후 09:02    <DIR>          embedding_fin
2020-08-30  오후 09:35             6,856 FE_NLP_MJ.py
2020-08-30  오후 09:41             6,865 FE_NLP_MJ_for_df.py
2020-08-30  오후 09:42            53,396 FE_NLP_test.ipynb
2020-08-28  오후 03:35           119,601 item_meta_v04_0828.xlsx
2020-08-30  오후 05:50             1,351 make_corpus_our.py
2020-08-30  오후 04:39               753 make_corpus_our.txt
2020-08-25  오후 12:36                67 requirement_NLP.bash
2020-08-28  오후 03:36         2,334,535 sale_data_v05_0828.xlsx
2020-08-30  오전 11:44        10,383,194 sale_FE_v01_0829.xlsx
2020-08-30  오후 09:09    <DIR>          tmp
               9개 파일          12,906,618 바이트
               6개 디렉터리  50,643,861,504 바이트 남음

## data 불러오기

단어사전을 만들 때, 빈도수를 고려해서 embedding으로 하려면, df_s = pd.read_excel("sale_data_v05_0828.xlsx") 사용

빈도수를 고려하지 않으려면, df_m = pd.read_excel("item_meta_v04_0828.xlsx") 사용

In [3]:
df_s = pd.read_excel("sale_data_v05_0828.xlsx")
df_m = pd.read_excel("item_meta_v04_0828.xlsx")

## 모듈(.py) 불러오기

#### make_corpus_our
    함수 두개~
    <함수설명>
    make_copus_our(df) : #우리가 나눈 브랜드/상품명다시/단위로 만든 토큰모음 (단어사전). df에 [브랜드][상품명다시][단위] column필수
    make_copus_M(df) : MeCab으로 만든 토큰모음 (단어사전)


In [4]:
import FE_make_corpus as MC #corpus만드는 모듈

In [5]:
corpus_our = MC.make_corpus_our(df_s) #우리가 나눈 브랜드/상품명다시/단위로 만든 토큰모음 (단어사전)
corpus_M = MC.make_corpus_M(df_s) # MeCab으로 만든 토큰모음 (단어사전)
#빈도수 고려 안하려면 df_m으로 
#corpus_our = MC.make_copus_our(df_m)
#corpus_M = MC.make_corpus_M(df_m)

## 모듈(.py) 불러오기

#### FE_NLP_MJ
    class FE_NLP : corpus에 있는 단어들을 W2V을 하고 상품명을 embedding하는 class (무형은 무조건 0)
    
    FE_NLP.FE_W2V(df, corpus, dim = 10, win = 3, min_cnt = 1)
    dim = W2V의 토큰의 벡터 차원
    win = W2V에서 window 사이즈
    min_cnt = corpus에서 최소 고려 횟수
    
    <함수설명>
    W2V(w2v_m = "skip") : corpus에 있는 단어를 word to vector. self로 지정된 dim, win, min_cnt사용. w2v_m = "skip" or "cbow"
    
    embedding version1 : ["NEW상품명"]을 Mecab으로 토크나이징 후, W2V한 vecotr를 concate. 
                        concate된 vector가 가장 긴 상품명에 맞춰 다른 vector들은 zero padding
                        !!make_copus_M로만든 corpus사용!!
                        
    embedding version2 : ["NEW상품명"]을 Mecab으로 토크나이징 후, W2V한 vecotr를 평균. embedding된 모든 vector 차원 == self.dim
                       !!make_copus_M로만든 corpus사용!!
                       
    embedding version3 : [브랜드][상품명다시][단위]로 토크나이징 후, W2V한 vecotr를 평균. embedding된 모든 vector 차원 == self.dim
                        !!make_copus_our로만든 corpus사용!!

In [6]:
import FE_NLP_MJ as FE_NLP #embedding하는 모듈

embedding version1,2 -> make_corpus_M으로 만든 corpus(여기서는 corpus_M)이 input

In [7]:
FE_NLP_test1 = FE_NLP.FE_W2V(df_s,corpus_M) # 1.객체지정 -> input은 df랑 corpus

In [8]:
FE_NLP_test1.W2V() #2. word2vec실행 (!꼭해주기!)

In [9]:
result_1 = FE_NLP_test1.product_name_embedding_ver1() #embedding version1 

In [10]:
result_1

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,866.0,테이트 여성 셀린니트 3종,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,866.0,테이트 여성 셀린니트 3종,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
result_2 = FE_NLP_test1.product_name_embedding_ver2() #embedding version2

In [12]:
result_2

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,...,0.018548,-0.004549,0.004717,0.026103,-0.014319,0.009187,0.024523,0.008044,-0.002186,0.006065
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,866.0,테이트 여성 셀린니트 3종,...,0.032990,-0.001584,0.016037,0.027920,-0.017705,0.008682,0.021820,0.013793,-0.005881,-0.000083
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,...,0.018548,-0.004549,0.004717,0.026103,-0.014319,0.009187,0.024523,0.008044,-0.002186,0.006065
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,866.0,테이트 여성 셀린니트 3종,...,0.032990,-0.001584,0.016037,0.027920,-0.017705,0.008682,0.021820,0.013793,-0.005881,-0.000083
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,...,0.018548,-0.004549,0.004717,0.026103,-0.014319,0.009187,0.024523,0.008044,-0.002186,0.006065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


embedding version3 -> make_corpus_our으로 만든 corpus(여기서는 corpus_our)이 input

In [13]:
FE_NLP_test2 = FE_NLP.FE_W2V(df_s,corpus_our) # 1.객체지정 -> input은 df랑 corpus

In [14]:
FE_NLP_test2.W2V() #2. word2vec실행 (!꼭해주기!)

In [15]:
result_3 = FE_NLP_test2.product_name_embedding_ver3() #embedding version3

In [16]:
result_3

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,...,0.010907,0.010153,0.029963,-0.014512,0.013113,0.002260,0.001445,0.018737,0.037791,-0.003868
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,866.0,테이트 여성 셀린니트 3종,...,0.025635,0.013361,0.015987,0.001411,0.008098,0.005843,0.003983,0.031757,0.034938,-0.002623
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,...,0.010907,0.010153,0.029963,-0.014512,0.013113,0.002260,0.001445,0.018737,0.037791,-0.003868
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,866.0,테이트 여성 셀린니트 3종,...,0.025635,0.013361,0.015987,0.001411,0.008098,0.005843,0.003983,0.031757,0.034938,-0.002623
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,...,0.010907,0.010153,0.029963,-0.014512,0.013113,0.002260,0.001445,0.018737,0.037791,-0.003868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### 사용예시

In [17]:
import make_corpus_our as makeC
import FE_NLP_MJ as NLP 

df = pd.read_excel("sale_data_v05_0828.xlsx")

corpus_our = makeC.make_corpus_our(df)
FE_NLP_fin = NLP.FE_W2V(df,corpus_our) # 1.객체지정 -> input은 df랑 corpus
FE_NLP_fin.W2V() #2. word2vec실행 (!꼭해주기!)
df_new = FE_NLP_fin.product_name_embedding_ver3() #3. embedding해서 다시 df로 지정 (embedding version3)

In [18]:
df_new

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,865.0,테이트 남성 셀린니트 3종,...,0.010907,0.010153,0.029963,-0.014512,0.013113,0.002260,0.001445,0.018737,0.037791,-0.003868
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,866.0,테이트 여성 셀린니트 3종,...,0.025635,0.013361,0.015987,0.001411,0.008098,0.005843,0.003983,0.031757,0.034938,-0.002623
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,865.0,테이트 남성 셀린니트 3종,...,0.010907,0.010153,0.029963,-0.014512,0.013113,0.002260,0.001445,0.018737,0.037791,-0.003868
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,866.0,테이트 여성 셀린니트 3종,...,0.025635,0.013361,0.015987,0.001411,0.008098,0.005843,0.003983,0.031757,0.034938,-0.002623
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,865.0,테이트 남성 셀린니트 3종,...,0.010907,0.010153,0.029963,-0.014512,0.013113,0.002260,0.001445,0.018737,0.037791,-0.003868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
